In [3]:
import os
import sys
from pathlib import Path
import numpy as np
from scipy.ndimage import center_of_mass
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np
from superpose3d import Superpose3D
PIPELINE_ROOT = Path('../src').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama

/home/eddyod/programming/preprocessing-pipeline/src


In [ ]:
def brain_to_atlas_transform(brain_coord, r, t):
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    atlas_coord = r @ brain_coord + t
    return atlas_coord.T[0] # Convert back to a row vector

def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))

def label_left_right(row):
    val = 'Singular'
    if str(row['Structure']).endswith('L'):
        val = 'Left'
    if str(row['Structure']).endswith('R'):
        val = 'Right'
    return val        

In [ ]:
animal = 'Atlas'
brain = BrainStructureManager(animal)
brain.fixed_brain = BrainStructureManager('Allen')

In [ ]:
brainstem_keys = set(brain.allen_structures_keys) - brain.midbrain_keys 

In [ ]:
moving_coms = brain.get_coms(annotator_id=1)
allen_coms = brain.fixed_brain.get_coms(annotator_id=1)
common_keys = allen_coms.keys() & moving_coms.keys() &  brain.midbrain_keys
brain_regions = sorted(moving_coms.keys())
allen_points = np.array([allen_coms[s] for s in brain_regions if s in common_keys])
moving_points = np.array([moving_coms[s] for s in brain_regions if s in common_keys])
allen_point_dict = {s:allen_coms[s] for s in brain_regions if s in common_keys}
moving_point_dict = {s:moving_coms[s] for s in brain_regions if s in common_keys}
assert len(moving_point_dict) > 0

In [ ]:
R, t = umeyama(moving_points.T, allen_points.T)

In [ ]:
t

In [ ]:
reg_points = R @ moving_points.T + t

In [ ]:
reg_point_dict = {s:reg_points.T[i] for i,s in enumerate(brain_regions) if s in common_keys}

In [ ]:
moving_point_dict

 That does not preclude starting the new idea but is just to say that somehow we need to get David's attention on what you have done- do a graph
= box plot of the midbrain com distances vs allen (active atlas based)
=box plot of the brainstem vs allen distances (but not using midbrain values to improve the distances)
This should provide an easy visual...


When the tethering is done for the brainstem structures though- he does not want the midbrain metrics to be included ( I don't know though if he means that the alignment is done with midbrain and then the evaluated outputs are only for the brainstem structures...?) your guess is best
So- in any case it was clear that he did not want metrics for midbrain and brainstem structures to be combined.

In [4]:
csvpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV8/csv'
df_midbrain_distance = pd.read_csv(os.path.join(csvpath, 'midbrain_distance.csv'))
df_midbrain_distance.head()

,Structure,distance
0,IC,101.584250
1,PBG_R,102.240882
2,SC,104.035306
3,SNR_R,165.298125
4,4N_L,171.430192


In [ ]:
distances = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    reg_point = brain_to_atlas_transform(moving_point, R, t)
    d = calculate_distance(allen_point, reg_point)
    distances.append(d)
    sortme[structure] = d

print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    print(f'<tr><td>{structure}</td><td>{round(d,2)}</td></tr>')

In [ ]:
df_distance = pd.DataFrame(ds.items(), columns=['Structure', 'distance'])
df_distance.head()

In [ ]:
fig, ax = plt.subplots()
columns = [distances]
ax.boxplot(columns, patch_artist=True)
plt.xticks([1], ["Registered"])
plt.show()

In [ ]:
df_allen = pd.DataFrame(allen_point_dict.items(), columns=['Structure', 'xyz'])
df_allen['S'] = df_allen.apply (lambda row: label_left_right(row), axis=1)
df_allen[['X', 'Y', 'Z']] = pd.DataFrame(df_allen['xyz'].tolist(), index=df_allen.index)

In [ ]:
df_atlas = pd.DataFrame(reg_point_dict.items(), columns=['Structure', 'xyz'])
df_atlas['S'] = df_atlas.apply (lambda row: label_left_right(row), axis=1)
df_atlas[['X', 'Y', 'Z']] = pd.DataFrame(df_atlas['xyz'].tolist(), index=df_atlas.index)

In [ ]:
df_atlas.head()

In [ ]:
xmin = 8000
xmax = 13000
ymin = 2000
ymax = 6000
zmin = 0
zmax = 25*456

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_allen[df_allen.S == label]
    atlas_subset = df_atlas[df_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas X Y scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_allen[df_allen.S == label]
    atlas_subset = df_atlas[df_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Allen/Altas X Z scatter plot')    
plt.legend()

In [ ]:
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_allen[df_allen.S == label]
    atlas_subset = df_atlas[df_atlas.S == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Allen/Altas Y Z scatter plot')    
plt.legend()